In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
hexagon = "M0,-2.3094010768L2,-1.1547005384 2,1.1547005384 0,2.3094010768 -2,1.1547005384 -2,-1.1547005384Z"

In [3]:
converter = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/converter.csv"
state_data = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/state_data.csv"
state_coords = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/cartogram_coords.csv"
hourglass_coords = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/hourglass_coords.csv"
district_data = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/app_building/app_data/district_data.csv"

In [4]:
cv = pd.read_csv(converter)
st = pd.read_csv(state_data)
sc = pd.read_csv(state_coords)
hc = pd.read_csv(hourglass_coords)
dd = pd.read_csv(district_data)

In [5]:
st = st.merge(sc, how="left")

## Experiment with Sliders

In [6]:
#build out the chart
df = st
df["ST"] = df["ST"]
slider = alt.binding_range(min=2004, max=2022, step=2)
select_year = alt.selection_single(name="Election", fields=['year'],
                                   bind=slider, init={'year': 2004})
#build the chare
st_cart = alt.Chart(df).mark_point().encode(
    alt.X('X',axis=None),
    alt.Y('Y',axis=None),
    alt.Text('ST'),
    alt.Color('porp_loss', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(title="Porportion not Represented")),
    tooltip = [alt.Tooltip('porp_text',title='Porportion Lost'),
               alt.Tooltip('dist_text',title='District Deficit')
              ]
)
#inser labels
st_cart_labs = alt.Chart(df).mark_text(align='center').encode(
    alt.X('X'),
    alt.Y('Y'),
    alt.Text('ST'),
).properties(title='Severity of Partisan Misrepresentation')
#display
alt.layer(st_cart, st_cart_labs).configure_axis(grid=False).configure_view(strokeWidth=0).configure_point(size=250,shape=hexagon,filled=True).add_selection(select_year).transform_filter(select_year)

alt.LayerChart(...)

# create a function to generate plots

In [7]:
#presets from formula
year = 2020
wave = "whatever"
#build out the chart
df = st[st["year"] == year]
df["ST"] = df["ST"]

<ipython-input-7-3690638fa34a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ST"] = df["ST"]


In [8]:
#state cartogram
#build the chart
st_cart = alt.Chart(df).mark_point().encode(
    alt.X('X',axis=None),
    alt.Y('Y',axis=None),
    alt.Text('ST'),
    alt.Color('porp_loss', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(title="Porportion not Represented")),
    tooltip = [alt.Tooltip('porp_text',title='Porportion Lost'),
               alt.Tooltip('dist_text',title='District Deficit')
              ]
)
#inser labels
st_cart_labs = alt.Chart(df).mark_text(align='center').encode(
    alt.X('X'),
    alt.Y('Y'),
    alt.Text('ST'),
).properties(title='Severity of Partisan Misrepresentation')
#display
alt.layer(st_cart, st_cart_labs).configure_axis(grid=False).configure_view(strokeWidth=0).configure_point(size=250,shape=hexagon,filled=True)

alt.LayerChart(...)

In [9]:
#create a function to change the probabilities based on a partisan wave

def call_wave(wave):
    '''Call a model by function name (to be used within a larger function)
    
    Args:
        A string that refers to the model name. Options are Linear SVC, SVC, and Logistic Regression 
    
    Returns:
        A model as a function for later use
    '''
    #create an if/else to identify the possible models
    if wave == "Republican":
        col = cv["logit_red"]
    elif wave == "Democratic":
        col = cv["logit_blue"]
    #return a string if it fails- it will block the function from applying
    else:
        col = cv["logit_all"]
    index = cv[["metric"]]
    index["prob"] = col
    return index

In [10]:
conv = call_wave(wave)
dist = dd[dd["year"] == year]
dist = dist.merge(conv, how="left")
#create the amounts chart for the Majority Bar
amounts = pd.DataFrame(dist.prob.value_counts()).reset_index()
amounts.columns = ["prob_GOP","num_dist"]
amounts["quantity"] = amounts["prob_GOP"] * amounts["num_dist"]
expec_GOP = amounts["quantity"].sum()
expec_dem = 435 - expec_GOP
d = {'Expected Seats' : [expec_GOP, expec_dem], 'Party' : ["GOP","Dem"]}
expec = pd.DataFrame(data = d, index=None)
#create the media seate hexmap

<ipython-input-9-56a58d351697>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index["prob"] = col


,Expected Seats,Party
0,219.0256,GOP
1,215.9744,Dem


In [11]:
#output majority control chart
cd_scale = [0,50,100,150,175,200,218,235,260,285,335,385,435]
alt.Chart(expec).mark_bar().encode(
    alt.X('sum(Expected Seats)',axis=alt.Axis(values=cd_scale)),
    alt.Color('Party')
)

alt.Chart(...)

In [ ]:
#
st_cart = alt.Chart(df).mark_point().encode(
    alt.X('X',axis=None),
    alt.Y('Y',axis=None),
    alt.Text('ST'),
    alt.Color('porp_loss', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(title="Porportion not Represented")),
    tooltip = [alt.Tooltip('porp_text',title='Porportion Lost'),
               alt.Tooltip('dist_text',title='District Deficit')
              ]
)

In [12]:
#build the hex cartogram
cv

,metric,pvi_range,prob_GOP_bin,prob_GOP_all,prob_GOP_red,prob_GOP_blue,logit_all,logit_red,logit_blue
0,0.06,D+26+,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
1,0.07,D+26+,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
2,0.08,D+26+,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
3,0.09,D+26+,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
4,0.10,D+26+,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...
84,0.90,R+21+,1.0,1.0,1.0,1.0,1.0,1.0,0.9999
85,0.91,R+21+,1.0,1.0,1.0,1.0,1.0,1.0,0.9999
86,0.92,R+21+,1.0,1.0,1.0,1.0,1.0,1.0,1.0000
87,0.93,R+21+,1.0,1.0,1.0,1.0,1.0,1.0,1.0000
